In [26]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt

# Importing sector DFs

In [27]:
# importing actual energy generation
PATH_TO_DATA = '/Users/R/code/HaukeFock/predict_energy_generation/raw_data/'
companies = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/

## Actual Energy Generation

In [37]:
columns = ['Date', 'Time of day', 'Biomass[MWh]', 'Hydropower[MWh]',
       'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]',
       'Other renewable[MWh]', 'Nuclear[MWh]', 'Fossil brown coal[MWh]',
       'Fossil hard coal[MWh]', 'Fossil gas[MWh]', 'Hydro pumped storage[MWh]',
       'Other conventional[MWh]']
dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company} = pd.DataFrame(columns=columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/energy_generation/{company}/{file}', sep=';')
        exec(f"df_{company} = df_{company}.append(data, ignore_index=True)")
    exec(f"dataframes['df_{company}'] = df_{company}")
    print(f'df_{company} Loaded!!!')

/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,4,6,9,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_50hertz Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (2,3,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_amprion Loaded!!!
df_tennet Loaded!!!


/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/R/.pyenv/versions/3.8.6/envs/predict_energy_generation/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


df_transnet_bw Loaded!!!


In [38]:
# dropping unecessary columns 
df_50hertz['Date'] = df_50hertz['Date'][:140448].append(df_50hertz['Datum'][140448:210720]).append(df_50hertz['Date'][210720:])
df_50hertz['Time of day'] = df_50hertz['Time of day'][:140448].append(df_50hertz['Uhrzeit'][140448:210720]).append(df_50hertz['Time of day'][210720:])

df_tennet['Date'] = df_tennet['Datum'][:70272].append(df_tennet['Date'][70272:])
df_tennet['Time of day'] = df_tennet['Uhrzeit'][:70272].append(df_tennet['Time of day'][70272:])

df_50hertz.drop(columns=['Datum', 'Uhrzeit'], inplace=True)
df_tennet.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in dataframes.items():
    if (df.columns != columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

>>>>>>>>>>>>>> df_50hertz <<<<<<<<<<<<<<

Shape: (226748, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]                0
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                 226748
Fossil brown coal[MWh]            0
Fossil hard coal[MWh]             0
Fossil gas[MWh]                   0
Hydro pumped storage[MWh]         0
Other conventional[MWh]           0
dtype: int64

>>>>>>>>>>>>>> df_amprion <<<<<<<<<<<<<<

Shape: (226748, 14)

NaN values: 
Date                              0
Time of day                       0
Biomass[MWh]                      0
Hydropower[MWh]                   0
Wind offshore[MWh]           226748
Wind onshore[MWh]                 0
Photovoltaics[MWh]                0
Other renewable[MWh]              0
Nuclear[MWh]                      0
Fossil

In [39]:
numeric_values = ['Biomass[MWh]',
                 'Hydropower[MWh]',
                 'Wind offshore[MWh]',
                 'Wind onshore[MWh]',
                 'Photovoltaics[MWh]',
                 'Other renewable[MWh]',
                 'Nuclear[MWh]',
                 'Fossil brown coal[MWh]',
                 'Fossil hard coal[MWh]',
                 'Fossil gas[MWh]',
                 'Hydro pumped storage[MWh]',
                 'Other conventional[MWh]']

def set_to_numeric(dataframes, numeric_values):
    # sets the features to numerical values
    for column in numeric_values:
        #print(column)
        for key, df in dataframes.items():
            #print(f'{key} => {column}')
            df[column] = df[column].apply(lambda x: str(x).replace(',',''))
            df[column] = pd.to_numeric(df[column], errors='coerce')

set_to_numeric(dataframes, numeric_values)

In [40]:
# join 'Date' and 'Time of Day' into one
def set_datetime(dataframes):
    for df in dataframes.keys():
        print(f'Working on {df}')
        exec(f"{df}['Date'] = pd.to_datetime({df}['Date']).apply(lambda x: x.strftime('%Y-%m-%d '))")
        exec(f"{df}['Time of day'] = pd.to_datetime({df}['Time of day']).apply(lambda x: x.strftime('%H:%M'))")
        exec(f"{df}['Date'] = pd.to_datetime({df}['Date'] + {df}['Time of day'])")
        exec(f"{df}.drop(columns=['Time of day'], inplace=True)")
set_datetime(dataframes)

Working on df_50hertz
Working on df_amprion
Working on df_tennet
Working on df_transnet_bw


## Forcasted Energy generation

In [ ]:
# importing actual energy generation
forecasted_columns = ['Date', 'Time of day', 'Total[MWh]', 'Wind offshore[MWh]', 
           'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'Other[MWh]']
forecasted_dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company}_forecasted = pd.DataFrame(columns=forecasted_columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/forcasted_generation/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/forcasted_generation/{company}/{file}', sep=';')
        exec(f"df_{company}_forecasted = df_{company}_forecasted.append(data, ignore_index=True)")
    exec(f"forecasted_dataframes['df_{company}_forecasted'] = df_{company}_forecasted")
    print(f'df_{company}_forecasted Loaded!!!')

In [ ]:
# dropping unecessary columns
df_tennet_forecasted['Date'] = df_tennet_forecasted.Date[:210720].append(df_tennet_forecasted.Datum[210720:222236])\
                                                                .append(df_tennet_forecasted.Date[222236:])


df_tennet_forecasted['Time of day'] = df_tennet_forecasted['Time of day'][:210720].append(df_tennet_forecasted.Uhrzeit[210720:222236])\
                                                          .append(df_tennet_forecasted['Time of day'][222236:]).isna().sum()
df_tennet_forecasted.drop(columns=['Datum', 'Uhrzeit'], inplace=True)

# missing values resume for each area
for key, df in forecasted_dataframes.items():
    if (df.columns != forecasted_columns).all():
        print(f'{key} has different columns than expected!')
    else:
        print(f'>>>>>>>>>>>>>> {key} <<<<<<<<<<<<<<\n')
        print(f'Shape: {df.shape}\n')
        print(f'NaN values: \n{df.isna().sum()}\n')

In [ ]:
# setting columns to numeric
forecasted_numeric_columns = ['Total[MWh]','Wind offshore[MWh]','Wind onshore[MWh]',
                            'Photovoltaics[MWh]','Other[MWh]']

set_to_numeric(forecasted_dataframes, forecasted_numeric_columns)

# join 'Date' and 'Time of Day' into one
set_datetime(forecasted_dataframes)

## Production capacity

In [ ]:
capacity_columns = ['Date', 'Time of day', 'Biomass[MW]', 'Hydropower[MW]',
       'Wind offshore[MW]', 'Wind onshore[MW]', 'Photovoltaics[MW]',
       'Other renewable[MW]', 'Nuclear[MW]', 'Fossil brown coal[MW]',
       'Fossil hard coal[MW]', 'Fossil gas[MW]', 'Hydro pumped storage[MW]',
       'Other conventional[MW]']
capacity_dataframes = {}

for company in companies:
    #print(company)
    exec(f"df_{company}_capacity = pd.DataFrame(columns=capacity_columns)")
    data_paths = !ls /Users/R/code/HaukeFock/predict_energy_generation/raw_data/energy_capacity/{company}
    for file in data_paths:
        #print(file)
        data = pd.read_csv(f'{PATH_TO_DATA}/energy_capacity/{company}/{file}', sep=';')
        exec(f"df_{company}_capacity = df_{company}_capacity.append(data, ignore_index=True)")
    exec(f"capacity_dataframes['df_{company}_capacity'] = df_{company}_capacity")
    print(f'df_{company}_capacity Loaded!!!')

In [ ]:
# dropping duplicates
for df in capacity_dataframes.keys():
    exec(f"{df} = {df}.drop(index=[2,5,8]).reset_index(drop=True)")

In [ ]:
capacity_numeric_values = ['Biomass[MW]', 'Hydropower[MW]', 'Wind offshore[MW]', 'Wind onshore[MW]',
                           'Photovoltaics[MW]', 'Other renewable[MW]', 'Nuclear[MW]', 'Fossil brown coal[MW]',
                           'Fossil hard coal[MW]', 'Fossil gas[MW]', 'Hydro pumped storage[MW]', 
                           'Other conventional[MW]']

set_to_numeric(capacity_dataframes, capacity_numeric_values)

In [ ]:
set_datetime(capacity_dataframes)

# Group data

## Daily

In [32]:
#group energy generation per day 
for df in dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).sum().reset_index()")

In [ ]:
#group forcasted energy generation per day 
for df in forecasted_dataframes.keys():
    exec(f"{df} = {df}.groupby(by={df}['Date'].dt.date).sum().reset_index()")

## Hourly

In [41]:
#group energy generation per hour 
for df in dataframes.keys():
    exec(f"{df}.index = {df}.Date")
    exec(f"{df} = {df}.resample('h').sum().reset_index()")

In [ ]:
#group forcasted energy generation per hour 
for df in forecasted_dataframes.keys():
    exec(f"{df}.index = {df}.Date")
    exec(f"{df} = {df}.resample('h').sum().reset_index()")

# Creating complete DF

## Actual

In [53]:
df_deutschland = pd.DataFrame()

for df, company in zip(dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland = df_deutschland.append({df}, ignore_index=True)")

df_deutschland.shape

(226464, 14)

## Forcasted

In [ ]:
df_deutschland_forecasted = pd.DataFrame()

for df, company in zip(forecasted_dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland_forecasted = df_deutschland_forecasted.append({df}, ignore_index=True)")

df_deutschland_forecasted.shape

## Capacity

In [ ]:
df_deutschland_capacity = pd.DataFrame()

for df, company in zip(capacity_dataframes.keys(), companies):
        exec(f"{df}['region'] = company")
        exec(f"df_deutschland_capacity = df_deutschland_capacity.append({df}, ignore_index=True)")

df_deutschland_capacity.shape

# Saving DFs

## Daily

### Actual

In [34]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

dataframes['df_deutschland'] = df_deutschland

for df in dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/energy_generation_data/{df}.csv', index=False)")

In [36]:
df = pd.read_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/energy_generation_data/df_50hertz.csv')
df.tail()

,Date,Wind offshore[MWh],Wind onshore[MWh],Photovoltaics[MWh],region
2354,2021-06-12,18271.0,137535.0,46132.0,50hertz
2355,2021-06-13,16984.0,117049.0,59390.0,50hertz
2356,2021-06-14,3325.0,13351.0,93578.0,50hertz
2357,2021-06-15,4842.0,23084.0,72422.0,50hertz
2358,2021-06-16,526.0,17479.0,84581.0,50hertz


### Forecasted

In [ ]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

forecasted_dataframes['df_deutschland_forecasted'] = df_deutschland_forecasted

for df in forecasted_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/forecasted_generation_data/{df}.csv', index=False)")

### Capacity

In [ ]:
final_columns = ['Date', 'Wind offshore[MW]', 'Wind onshore[MW]', 'Photovoltaics[MW]', 'region']

capacity_dataframes['df_deutschland_capacity'] = df_deutschland_capacity

for df in capacity_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/daily/energy_capacity_data/{df}.csv', index=False)")
    

## Hourly

### Actual

In [25]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

dataframes['df_deutschland'] = df_deutschland

for df in dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/hourly/energy_generation_data/{df}.csv', index=False)")
    

### Forecasted

In [ ]:
final_columns = ['Date', 'Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]', 'region']

forecasted_dataframes['df_deutschland_forecasted'] = df_deutschland_forecasted

for df in forecasted_dataframes.keys():
    exec(f"{df}[final_columns].to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/hourly/forecasted_generation_data/{df}.csv', index=False)")

# scratch

In [57]:
#creating a total
df_deutschland.set_index('Date', inplace=True)

previous_year_df = df_deutschland.resample('h').sum()[['Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]']].loc['2020-1-1':'2020-12-31']

previous_year_df['Wind'] = previous_year_df[['Wind offshore[MWh]', 'Wind onshore[MWh]']].sum(axis=1)
previous_year_df['Solar'] = previous_year_df['Photovoltaics[MWh]']
previous_year_df['Total'] = previous_year_df.sum(axis=1)

previous_year_df.drop(columns=['Wind offshore[MWh]', 'Wind onshore[MWh]', 'Photovoltaics[MWh]'], inplace=True)

In [71]:
previous_year_df.to_csv('/Users/R/code/HaukeFock/predict_energy_generation/raw_data/hourly/energy_generation_data/total_energy_production_2020.csv')